This notebook is undocumented (and not cleanly coded) since it just gathers resources for the pdf document. Please refer to said document for results and explanations.

In [ ]:
import gc
import os.path
import pickle
import random
from typing import List
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from statsmodels.stats.contingency_tables import mcnemar


from Project.AutoSimilarityCacheConfiguration.DataAccess import DataAccess
from Project.Utils.ConfigurationUtils.ConfigLoader import ConfigLoader
from Project.Utils.Misc.OriginContainer import OriginContainer
from Project.VisualFeaturesBranches.ObjectDetection.ArtDataset import ArtDataset
from Project.VisualFeaturesBranches.ObjectDetection.DatasetSplits import DatasetSplits
from Project.VisualFeaturesBranches.ObjectDetection.ObjectDetectionNetworkUtils import collate, get_pretrained_model, train_one_epoch, load_model, reduce_box_count, get_winning_config_with_augmentations, get_winning_config_without_augmentations

cl: ConfigLoader = ConfigLoader.instance
cl.data_cleaning_in_progress = True  # Prohibits cache generation in DataAccess class

da: DataAccess = DataAccess.instance

check_ids_tree: List[str] = []
for i in da.get_ids_for_which_tree_annotation_exists():
    if i not in da.get_ids_for_which_bounding_boxes_exist():
        check_ids_tree.append(i)

check_ids_person: List[str] = []
for i in da.get_ids_for_which_person_annotation_exists():
    if i not in da.get_ids_for_which_bounding_boxes_exist():
        check_ids_person.append(i)

if not os.path.exists('result_dict_trees.pkl'):
    with open('result_dict_trees.pkl', 'wb+') as f:
        pickle.dump(dict(), f)
if not os.path.exists('result_dict_persons.pkl'):
    with open('result_dict_persons.pkl', 'wb+') as f:
        pickle.dump(dict(), f)
with open('result_dict_trees.pkl', 'rb') as f:
    tree_result_dict = pickle.load(f)
with open('result_dict_persons.pkl', 'rb') as f:
    persons_result_dict = pickle.load(f)

number_of_trainable_layers = 5

In [ ]:
def get_dataset_splits():
    return DatasetSplits(1200, 0, 0, 0, 0, 0, (0, 0), 0, False)

def get_art_dataset(ids):
    return ArtDataset(ids, 1200, 1200, 0, 0, 0, 0, 0, 0, (0, 0), 0)

In [ ]:
dss: DatasetSplits = get_dataset_splits()
len_train_set = sum(lengths := [len(ds) for ds in [dss.get_train(nr, False, None) for nr in range(dss.number_of_splits)]])/len(lengths)
len_test_set = sum(lengths := [len(ds) for ds in [dss.get_test(nr) for nr in range(dss.number_of_splits)]])/len(lengths)

In [ ]:
dss: DatasetSplits = get_dataset_splits()

trees_in_train_positive = 0
trees_in_train_negative = 0
trees_in_test_positive = 0
trees_in_test_negative = 0
trees_in_all_positive = 0
trees_in_all_negative = 0

trees_in_train_splits_positive = []
trees_in_train_splits_negative = []
trees_in_test_splits_positive = []
trees_in_test_splits_negative = []
trees_in_all_splits_positive = []
trees_in_all_splits_negative = []


for nr in range(dss.number_of_splits):
    trees_in_train_splits_positive.append(0)
    trees_in_train_splits_negative.append(0)
    trees_in_test_splits_positive.append(0)
    trees_in_test_splits_negative.append(0)
    for e in iter(dss.get_train(nr, False, None)):
        if 'Baum' in [da.get_class_label_for_index(l) for l in e[1]['labels']]:
            trees_in_train_positive += 1
            trees_in_train_splits_positive[nr] += 1
        else:
            trees_in_train_negative += 1
            trees_in_train_splits_negative[nr] += 1
    for e in iter(dss.get_test(nr)):
        if 'Baum' in [da.get_class_label_for_index(l) for l in e[1]['labels']]:
            trees_in_test_positive += 1
            trees_in_test_splits_positive[nr] += 1
        else:
            trees_in_test_negative += 1
            trees_in_test_splits_negative[nr] += 1

trees_in_test_positive /= dss.number_of_splits
trees_in_test_negative /= dss.number_of_splits
trees_in_train_positive /= dss.number_of_splits
trees_in_train_negative /= dss.number_of_splits

for e in iter(dss.get_training_all(0, 1)):
    if 'Baum' in [da.get_class_label_for_index(l) for l in e[1]['labels']]:
        trees_in_all_positive += 1
    else:
        trees_in_all_negative += 1

positive = 0
negative = 0
for c in check_ids_tree:
    if da.get_tree_annotation_for_identifier(c):
        positive += 1
    else:
        negative += 1

trees_in_train_splits_positive = [p / len_train_set for p in trees_in_train_splits_positive]
trees_in_train_splits_negative = [p / len_train_set for p in trees_in_train_splits_negative]
trees_in_test_splits_positive = [p / len_test_set for p in trees_in_test_splits_positive]
trees_in_test_splits_negative = [p / len_test_set for p in trees_in_test_splits_negative]

print(
    f'tree annotations: {positive / (positive + negative)}\ntraining set: {trees_in_train_positive / (trees_in_train_positive + trees_in_train_negative)}\nvalidation set: {trees_in_test_positive / (trees_in_test_positive + trees_in_test_negative)}\nall training: {trees_in_all_positive / (trees_in_all_positive + trees_in_all_negative)}\n\nAll splits: {trees_in_train_splits_positive=}, {trees_in_train_splits_negative=}, {trees_in_test_splits_positive=}, {trees_in_test_splits_negative=}')

In [ ]:
dss: DatasetSplits = get_dataset_splits()

persons_in_train_positive = 0
persons_in_train_negative = 0
persons_in_test_positive = 0
persons_in_test_negative = 0
persons_in_all_positive = 0
persons_in_all_negative = 0

persons_in_train_splits_positive = []
persons_in_train_splits_negative = []
persons_in_test_splits_positive = []
persons_in_test_splits_negative = []
persons_in_all_splits_positive = []
persons_in_all_splits_negative = []

for nr in range(dss.number_of_splits):
    persons_in_train_splits_positive.append(0)
    persons_in_train_splits_negative.append(0)
    persons_in_test_splits_positive.append(0)
    persons_in_test_splits_negative.append(0)

    for e in iter(dss.get_train(nr, False, None)):
        if 'Person' in [da.get_class_label_for_index(l) for l in e[1]['labels']]:
            persons_in_train_positive += 1
            persons_in_train_splits_positive[nr] += 1
        else:
            persons_in_train_negative += 1
            persons_in_train_splits_negative[nr] += 1
    for e in iter(dss.get_test(nr)):
        if 'Person' in [da.get_class_label_for_index(l) for l in e[1]['labels']]:
            persons_in_test_positive += 1
            persons_in_test_splits_positive[nr] += 1
        else:
            persons_in_test_negative += 1
            persons_in_test_splits_negative[nr] += 1

persons_in_test_positive /= dss.number_of_splits
persons_in_test_negative /= dss.number_of_splits
persons_in_train_positive /= dss.number_of_splits
persons_in_train_negative /= dss.number_of_splits

for e in iter(dss.get_training_all(0, 1)):
    if 'Person' in [da.get_class_label_for_index(l) for l in e[1]['labels']]:
        persons_in_all_positive += 1
    else:
        persons_in_all_negative += 1

positive = 0
negative = 0
for c in check_ids_person:
    if da.get_person_annotation_for_identifier(c):
        positive += 1
    else:
        negative += 1

persons_in_train_splits_positive = [p / len_train_set for p in persons_in_train_splits_positive]
persons_in_train_splits_negative = [p / len_train_set for p in persons_in_train_splits_negative]
persons_in_test_splits_positive = [p / len_test_set for p in persons_in_test_splits_positive]
persons_in_test_splits_negative = [p / len_test_set for p in persons_in_test_splits_negative]

print(
    f'person annotations: {positive / (positive + negative)}\ntraining set: {persons_in_train_positive / (persons_in_train_positive + persons_in_train_negative)}\nvalidation set: {persons_in_test_positive / (persons_in_test_positive + persons_in_test_negative)}\nall training: {persons_in_all_positive / (persons_in_all_positive + persons_in_all_negative)}\n\nAll splits: {persons_in_train_splits_positive=}, {persons_in_train_splits_negative=}, {persons_in_test_splits_positive=}, {persons_in_test_splits_negative=}')

In [ ]:
model_count = 0
while True:
    if os.path.exists(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 'VisualFeaturesBranches',
                                   'RegionOfInterestExtractor', 'Generated_Files', 'State_Dictionary',
                                   f'best_config_grid_{model_count + 1}_model.pkl')):
        model_count += 1
    else:
        break


def get_stats(from_model, ds, device, score_threshold, for_tree, nms=0.4):
    if for_tree:
        check_ids = check_ids_tree
    else:
        check_ids = check_ids_person
    from_model.to(device)
    from_model.eval()
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    pos = 0
    neg = 0
    identifiers_false_positive = []
    identifiers_false_negative = []

    for ident in (loading_bar := tqdm(check_ids, total=len(check_ids))):
        stats_img, _ = ds.get_item_by_identifier(ident)
        stats_output = from_model([stats_img.to(device)])
        stats_reduced = reduce_box_count(stats_output[0], nms, score_threshold, 0)
        stats_labels = [da.get_class_label_for_index(l) for l in stats_reduced['labels']]

        del stats_img
        del stats_output
        del stats_reduced
        gc.collect()

        if for_tree:
            found = 'Baum' in stats_labels
            exists = da.get_tree_annotation_for_identifier(ident)
        else:
            found = 'Person' in stats_labels
            exists = da.get_person_annotation_for_identifier(ident)

        if exists:
            pos += 1
            if found:
                true_positive += 1
            else:
                false_negative += 1
                identifiers_false_negative.append(ident)
        else:
            neg += 1
            if found:
                false_positive += 1
                identifiers_false_positive.append(ident)
            else:
                true_negative += 1
        loading_bar.set_description(
            f'{true_positive=}, {false_positive=}, {true_negative=}, {false_negative=}, {pos=}, {neg=}')
    return true_positive, false_positive, true_negative, false_negative, pos, neg, identifiers_false_positive, identifiers_false_negative


for experiment, config in zip(['Without_Augmentations', 'With_Augmentations'], [get_winning_config_without_augmentations()[0], get_winning_config_with_augmentations()[0]]):
    if experiment in tree_result_dict.keys() and experiment in persons_result_dict.keys():
        continue
    print(f'Testing model from experiment {experiment}')

    model = load_model(experiment)

    threshold = 0.4

    if experiment not in tree_result_dict.keys():
        tree_result_dict[experiment] = (get_stats(model, get_art_dataset(check_ids_tree), config['device'], threshold, True))
    if experiment not in persons_result_dict.keys():
        persons_result_dict[experiment] = (
            get_stats(model, get_art_dataset(check_ids_person), config['device'], threshold, False))
    del model
    gc.collect()
    with open('result_dict_trees.pkl', 'wb+') as f:
        pickle.dump(tree_result_dict, f)
    with open('result_dict_persons.pkl', 'wb+') as f:
        pickle.dump(persons_result_dict, f)

In [ ]:
def plot_confusion_matrix(matrix_entry, name):
    true_positive = matrix_entry[0]
    false_positive = matrix_entry[1]
    true_negative = matrix_entry[2]
    false_negative = matrix_entry[3]
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    f1 = (2 * precision * recall) / (precision + recall)

    plt.subplots()
    sn.heatmap(pd.DataFrame([[true_positive, false_positive], [false_negative, true_negative]],
                            index=['Predicted Positive', 'Predicted Negative'],
                            columns=['True Positive', 'True Negative']), annot=True, vmin=0, vmax=1000, fmt="",
               annot_kws={'weight': 'bold', 'size': 'large'})
    plt.title(f'F1: {f1:1.3f}, Precision: {precision:1.3f},  Recall: {recall:1.3f}', fontdict={'fontsize': 14})

    plt.savefig(f'confusion_matrix_{name}.png')

for k in tree_result_dict.keys():
    entry = tree_result_dict[k]
    plot_confusion_matrix(entry, f'{k}_tree')

In [ ]:
for k in persons_result_dict.keys():
    entry = persons_result_dict[k]
    plot_confusion_matrix(entry, f'{k}_person')

In [ ]:
best_experiment = 'With_Augmentations'
best_model = load_model(best_experiment)
best_config = get_winning_config_with_augmentations()[0]
best_size = best_config['size']
best_horizontal_flip_p = best_config['p_horizontal_flip']
best_vertical_flip_p = best_config['p_vertical_flip']
best_brightness_jitter = best_config['brightness_jitter']
best_contrast_jitter = best_config['contrast_jitter']
best_saturation_jitter = best_config['saturation_jitter']
best_hue_jitter = best_config['hue_jitter']
best_p_grayscale = best_config['p_grayscale']


def get_training_data_set(percentage, seed):
    data_access: DataAccess = DataAccess.instance
    identifiers = list(data_access.get_ids_for_which_bounding_boxes_exist())
    random.seed(seed)
    random.shuffle(identifiers)
    identifiers = identifiers[:int(len(identifiers) * percentage)]
    return identifiers


def evaluate_model_for_dataset(model_to_evaluate, score_threshold, device, for_tree, nms):
    if for_tree:
        check_ids = check_ids_tree
    else:
        check_ids = check_ids_person
    stats = get_stats(model_to_evaluate, get_art_dataset(check_ids), device, score_threshold, for_tree, nms=nms)
    true_positive = stats[0]
    false_positive = stats[1]
    false_negative = stats[3]
    if (true_positive + false_positive) == 0 or (true_positive + false_negative) == 0:
        return 0
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    if precision + recall == 0:
        return 0
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def get_model_for_evaluation(configuration, training_ids, train_for_epochs):
    lr, weight_decay, trainable_layers, size, p_horizontal_flip, p_vertical_flip, brightness_jitter, saturation_jitter, hue_jitter, p_grayscale, batch_size, num_workers, device, average_span, nr_epochs, report, small_model, curriculum = configuration['lr'], configuration['weight_decay'], configuration['trainable_layers'], configuration['size'], configuration['p_horizontal_flip'], configuration['p_vertical_flip'], configuration['brightness_jitter'], configuration['saturation_jitter'], configuration['hue_jitter'], configuration['p_grayscale'], configuration['batch_size'], configuration['num_workers'], configuration['device'], configuration['average_span'], configuration['nr_epochs'], configuration['report'],  configuration['small_model'], configuration['curriculum']

    model_to_eval = get_pretrained_model(trainable_layers, device, small_model)
    model_to_eval.train()

    if len(training_ids) != 0:
        optimizer = torch.optim.Adam(params=[param for param in model_to_eval.parameters() if param.requires_grad], lr=lr,
                                     weight_decay=weight_decay)

        for epoch in range(train_for_epochs):
            factor = 1
            if curriculum:
                factor = epoch / train_for_epochs
            training_dataset = ArtDataset(training_ids, width=best_size, height=best_size, for_training=True,
                          p_horizontal_flip=best_horizontal_flip_p * factor, p_vertical_flip=best_vertical_flip_p * factor,
                          brightness_jitter=best_brightness_jitter * factor, contrast_jitter=best_contrast_jitter * factor,
                          saturation_jitter=best_saturation_jitter * factor, hue_jitter=best_hue_jitter * factor,
                          p_grayscale=best_p_grayscale * factor)
            training_loader = DataLoader(training_dataset, batch_size=batch_size, collate_fn=collate,
                                         shuffle=True, num_workers=num_workers, drop_last=True)
            model_to_eval = train_one_epoch(model_to_eval, optimizer, training_loader, epoch, device, average_span, report)
            del training_dataset
            gc.collect()
    return model_to_eval

In [ ]:
if not os.path.exists('best_nms_and_result_tree.pkl'):
    best_nms_tree = None
    best_result_tree = 0

    for t in [0.05 * i for i in range(0, 21)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset(best_model, score_threshold=0, device=best_config['device'],
                                                   for_tree=True, nms=t)
        if current_score > best_result_tree:
            best_result_tree = current_score
            best_nms_tree = t
    with open('best_nms_and_result_tree.pkl', 'wb+') as f:
        pickle.dump((best_nms_tree, best_result_tree), f)
else:
    with open('best_nms_and_result_tree.pkl', 'rb') as f:
        best_nms_tree, best_result_tree = pickle.load(f)
print(f'{best_nms_tree=}, {best_result_tree=}')

if not os.path.exists('best_threshold_and_result_tree.pkl'):
    best_threshold_tree = None
    best_result_tree = 0

    for t in [0.05 * i for i in range(0, 10)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset(best_model, score_threshold=t, device=best_config['device'],
                                                   for_tree=True, nms=best_nms_tree)
        if current_score > best_result_tree:
            best_result_tree = current_score
            best_threshold_tree = t
    with open('best_threshold_and_result_tree.pkl', 'wb+') as f:
        pickle.dump((best_threshold_tree, best_result_tree), f)
else:
    with open('best_threshold_and_result_tree.pkl', 'rb') as f:
        best_threshold_tree, best_result_tree = pickle.load(f)
print(f'{best_threshold_tree=}, {best_result_tree=}')

In [ ]:
if not os.path.exists('best_nms_and_result_person.pkl'):
    best_nms_person = None
    best_result_person = 0

    for t in [0.05 * i for i in range(0, 21)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset(best_model, score_threshold=0, device=best_config['device'],
                                                   for_tree=False, nms=t)
        if current_score > best_result_person:
            best_result_person = current_score
            best_nms_person = t
    with open('best_nms_and_result_person.pkl', 'wb+') as f:
        pickle.dump((best_nms_person, best_result_person), f)
else:
    with open('best_nms_and_result_person.pkl', 'rb') as f:
        best_nms_person, best_result_person = pickle.load(f)
print(f'{best_nms_person=}, {best_result_person=}')

if not os.path.exists('best_threshold_and_result_person.pkl'):
    best_threshold_person = None
    best_result_person = 0

    for t in [0.05 * i for i in range(0, 13)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset(best_model, score_threshold=t, device=best_config['device'],
                                                   for_tree=False, nms=best_nms_person)
        if current_score > best_result_person:
            best_result_person = current_score
            best_threshold_person = t
    with open('best_threshold_and_result_person.pkl', 'wb+') as f:
        pickle.dump((best_threshold_person, best_result_person), f)
else:
    with open('best_threshold_and_result_person.pkl', 'rb') as f:
        best_threshold_person, best_result_person = pickle.load(f)
print(f'{best_threshold_person=}, {best_result_person=}')

In [ ]:
def plot_confusion_matrix_manual(tp, fp, tn, fn, save_as_name):
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)
    sn.heatmap(pd.DataFrame([[tp, fp], [fn, tn]], index=['Predicted Positive', 'Predicted Negative'],
                            columns=['True Positive', 'True Negative']), annot=True, vmin=0, vmax=1000, fmt="",
               annot_kws={'weight': 'bold', 'size': 'large'})
    plt.title(f'F1: {f1:1.3f}, Precision: {precision:1.3f},  Recall: {recall:1.3f}', fontdict={'fontsize': 14})

    plt.savefig(f'{save_as_name}.png')

In [ ]:
if not os.path.exists('best_model_stats_tree.pkl'):
    with open('best_model_stats_tree.pkl', 'wb+') as f:
        pickle.dump(get_stats(best_model, get_art_dataset(check_ids_tree), torch.device('cuda'), best_threshold_tree, True, best_nms_tree), f)
with open('best_model_stats_tree.pkl', 'rb') as f:
    best_model_stats_tree = pickle.load(f)
if not os.path.exists('best_model_stats_person.pkl'):
    with open('best_model_stats_person.pkl', 'wb+') as f:
        pickle.dump(get_stats(best_model, get_art_dataset(check_ids_person), torch.device('cuda'), best_threshold_person, False, best_nms_person), f)
with open('best_model_stats_person.pkl', 'rb') as f:
    best_model_stats_person = pickle.load(f)

In [ ]:
plot_confusion_matrix_manual(best_model_stats_tree[0], best_model_stats_tree[1], best_model_stats_tree[2], best_model_stats_tree[3], 'best_tree_confusion_matrix')

In [ ]:
plot_confusion_matrix_manual(best_model_stats_person[0], best_model_stats_person[1], best_model_stats_person[2], best_model_stats_person[3], 'best_person_confusion_matrix')

In [ ]:
del best_model
gc.collect()
best_nr_of_epochs = get_winning_config_with_augmentations()[1]

if not os.path.exists('tree_scores_dict.pkl'):
    with open('tree_scores_dict.pkl', 'wb+') as f:
        pickle.dump(dict(), f)
if not os.path.exists('person_scores_dict.pkl'):
    with open('person_scores_dict.pkl', 'wb+') as f:
        pickle.dump(dict(), f)

with open('tree_scores_dict.pkl', 'rb') as f:
    tree_scores_dict = pickle.load(f)
with open('person_scores_dict.pkl', 'rb') as f:
    person_scores_dict = pickle.load(f)

for p in [0.1 * i for i in range(0, 11)]:
    if p not in tree_scores_dict.keys() or p not in person_scores_dict.keys():
        print(f'testing with with {p}')
        tree_scores = []
        person_scores = []

        for s in range(5):
            current_model = get_model_for_evaluation(best_config, get_training_data_set(p, s), best_nr_of_epochs)
            tree_scores.append(evaluate_model_for_dataset(current_model, best_threshold_tree, best_config['device'], True, best_nms_tree))
            person_scores.append(evaluate_model_for_dataset(current_model, best_threshold_person, best_config['device'], False, best_nms_person))
            del current_model
            gc.collect()
        tree_scores_dict[p] = sum(tree_scores) / len(tree_scores)
        person_scores_dict[p] = sum(person_scores) / len(person_scores)

        with open('tree_scores_dict.pkl', 'wb+') as f:
            pickle.dump(tree_scores_dict, f)
        with open('person_scores_dict.pkl', 'wb+') as f:
            pickle.dump(person_scores_dict, f)

print(tree_scores_dict)
print(person_scores_dict)

In [ ]:
for current, plot_title, save_as in zip([tree_scores_dict, person_scores_dict],
                                        ['Tree Dataset F1 Scores for Model trained on x Percent of the Data',
                                         'Person Dataset F1 Scores for Model trained on x Percent of the Data'],
                                        ['tree_f1_scores_for_small_dataset.png',
                                         'person_f1_scores_for_small_dataset.png']):
    _, ax = plt.subplots()
    ax.bar(list(round(k, 1) * 10 for k in current.keys()), current.values())
    title = plot_title
    ax.set_title(title)
    ax.set_xlabel('Percentage of Data')
    ax.set_ylabel('F1 Score')
    ax.set_xticks(list(round(k, 1) * 10 for k in current.keys()), labels=[round(k, 1) for k in current.keys()])
    plt.savefig(save_as)

In [ ]:
da: DataAccess = DataAccess.instance
all_iconclass_labels = set()

both_positive = 0
only_iconclass_positive = 0
only_ground_truth_positive = 0
both_negative = 0

for i in tqdm(check_ids_tree):
    tree_in_iconclass = False

    for tag in ['Wald', 'Baumgruppen', 'Laubwald', 'Bäume', 'Lichtung im Wald', 'Waldrand', 'Waldweg']:
        if tag in da.get_iconclass_tags_from_identifier(i):
            tree_in_iconclass = True
            break

    ground_truth = da.get_tree_annotation_for_identifier(i)

    if tree_in_iconclass and ground_truth:
        both_positive += 1
    elif tree_in_iconclass and not ground_truth:
        only_iconclass_positive += 1
    elif ground_truth and not tree_in_iconclass:
        only_ground_truth_positive += 1
    elif not tree_in_iconclass and not ground_truth:
        both_negative += 1
hit = both_positive + both_negative
miss = only_iconclass_positive + only_ground_truth_positive
print(f'{both_positive=}, {only_iconclass_positive=}, {only_ground_truth_positive=}, {both_negative=}, {hit=}, {miss=}')

In [ ]:
da: DataAccess = DataAccess.instance
all_iconclass_labels = set()

both_positive = 0
only_tag_positive = 0
only_ground_truth_positive = 0
both_negative = 0

for i in tqdm(check_ids_tree):
    tree_in_tags = 'Baum' in [t[0] for t in da.get_tag_tuples_from_identifier(i, OriginContainer(('Obj',)))]

    ground_truth = da.get_tree_annotation_for_identifier(i)

    if tree_in_tags and ground_truth:
        both_positive += 1
    elif tree_in_tags and not ground_truth:
        only_tag_positive += 1
    elif ground_truth and not tree_in_tags:
        only_ground_truth_positive += 1
    elif not tree_in_tags and not ground_truth:
        both_negative += 1
hit = both_positive + both_negative
miss = only_tag_positive + only_ground_truth_positive
print(f'{both_positive=}, {only_tag_positive=}, {only_ground_truth_positive=}, {both_negative=}, {hit=}, {miss=}')

In [ ]:
da: DataAccess = DataAccess.instance
all_iconclass_labels = set()

both_positive = 0
only_prediction_positive = 0
only_ground_truth_positive = 0
both_negative = 0

dataset = get_art_dataset(check_ids_tree)

model = load_model('With_Augmentations')
model.eval()

for identifier in check_ids_tree:
    img, targets = dataset.get_item_by_identifier(identifier)
    output = model([img.to(torch.device('cuda'))])
    reduced = reduce_box_count(output[0], best_nms_tree, best_threshold_tree, 0)

    del img
    del output
    gc.collect()

    labels = [da.get_class_label_for_index(l) for l in reduced['labels']]
    tree_in_prediction = 'Baum' in labels

    ground_truth = da.get_tree_annotation_for_identifier(identifier)

    if tree_in_prediction and ground_truth:
        both_positive += 1
    elif tree_in_prediction and not ground_truth:
        only_prediction_positive += 1
    elif ground_truth and not tree_in_prediction:
        only_ground_truth_positive += 1
    elif not tree_in_prediction and not ground_truth:
        both_negative += 1
hit = both_positive + both_negative
miss = only_prediction_positive + only_ground_truth_positive
print(f'{both_positive=}, {only_prediction_positive=}, {only_ground_truth_positive=}, {both_negative=}, {hit=}, {miss=}')
del model
gc.collect()

In [ ]:
untrained_model = get_pretrained_model(number_of_trainable_layers, torch.device('cuda'), True)
untrained_model.train()

if not os.path.exists('best_nms_and_result_untrained_person.pkl'):
    best_nms_untrained_person = None
    best_result_untrained_person = 0

    for t in [0.05 * i for i in range(0, 21)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset(untrained_model, score_threshold=0, device=best_config['device'],
                                                   for_tree=False, nms=t)
        if current_score > best_result_untrained_person:
            best_result_untrained_person = current_score
            best_nms_untrained_person = t
    with open('best_nms_and_result_untrained_person.pkl', 'wb+') as f:
        pickle.dump((best_nms_untrained_person, best_result_untrained_person), f)
else:
    with open('best_nms_and_result_untrained_person.pkl', 'rb') as f:
        best_nms_untrained_person, best_result_untrained_person = pickle.load(f)
print(f'{best_nms_untrained_person=}, {best_result_untrained_person=}')

if not os.path.exists('best_threshold_and_result_untrained_person.pkl'):
    best_threshold_untrained_person = None
    best_result_untrained_person = 0

    for t in [0.05 * i for i in range(0, 11)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset(untrained_model, score_threshold=t, device=best_config['device'],
                                                   for_tree=False, nms=best_nms_untrained_person)
        if current_score > best_result_untrained_person:
            best_result_untrained_person = current_score
            best_threshold_untrained_person = t
    with open('best_threshold_and_result_untrained_person.pkl', 'wb+') as f:
        pickle.dump((best_threshold_untrained_person, best_result_untrained_person), f)
else:
    with open('best_threshold_and_result_untrained_person.pkl', 'rb') as f:
        best_threshold_untrained_person, best_result_untrained_person = pickle.load(f)
print(f'{best_threshold_untrained_person=}, {best_result_untrained_person=}')

In [ ]:
gc.collect()
untrained_model = get_pretrained_model(number_of_trainable_layers, torch.device('cuda'), True)
untrained_model.train()

if not os.path.exists('best_nms_and_result_untrained_tree.pkl'):
    best_nms_untrained_tree = None
    best_result_untrained_tree = 0

    for t in [0.05 * i for i in range(0, 21)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset(untrained_model, score_threshold=0, device=best_config['device'],
                                                   for_tree=True, nms=t)
        if current_score > best_result_untrained_tree:
            best_result_untrained_tree = current_score
            best_nms_untrained_tree = t
    with open('best_nms_and_result_untrained_tree.pkl', 'wb+') as f:
        pickle.dump((best_nms_untrained_tree, best_result_untrained_tree), f)
else:
    with open('best_nms_and_result_untrained_tree.pkl', 'rb') as f:
        best_nms_untrained_tree, best_result_untrained_tree = pickle.load(f)
print(f'{best_nms_untrained_tree=}, {best_result_untrained_tree=}')

if not os.path.exists('best_threshold_and_result_untrained_tree.pkl'):
    best_threshold_untrained_tree = None
    best_result_untrained_tree = 0

    for t in [0.05 * i for i in range(0, 11)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset(untrained_model, score_threshold=t, device=best_config['device'],
                                                   for_tree=True, nms=best_nms_untrained_tree)
        if current_score > best_result_untrained_tree:
            best_result_untrained_tree = current_score
            best_threshold_untrained_tree = t
    with open('best_threshold_and_result_untrained_tree.pkl', 'wb+') as f:
        pickle.dump((best_threshold_untrained_tree, best_result_untrained_tree), f)
else:
    with open('best_threshold_and_result_untrained_tree.pkl', 'rb') as f:
        best_threshold_untrained_tree, best_result_untrained_tree = pickle.load(f)
print(f'{best_threshold_untrained_tree=}, {best_result_untrained_tree=}')

In [ ]:
if not os.path.exists('untrained_model_stats_tree.pkl'):
    with open('untrained_model_stats_tree.pkl', 'wb+') as f:
        pickle.dump(get_stats(untrained_model, get_art_dataset(check_ids_tree), torch.device('cuda'), best_threshold_untrained_tree, True, best_nms_untrained_tree), f)
with open('untrained_model_stats_tree.pkl', 'rb') as f:
    untrained_model_stats_tree = pickle.load(f)
if not os.path.exists('untrained_model_stats_person.pkl'):
    with open('untrained_model_stats_person.pkl', 'wb+') as f:
        pickle.dump(get_stats(untrained_model, get_art_dataset(check_ids_person), torch.device('cuda'), best_threshold_untrained_person, False, best_nms_untrained_person), f)
with open('untrained_model_stats_person.pkl', 'rb') as f:
    untrained_model_stats_person = pickle.load(f)

In [ ]:
plot_confusion_matrix_manual(untrained_model_stats_tree[0], untrained_model_stats_tree[1], untrained_model_stats_tree[2], untrained_model_stats_tree[3], 'untrained_tree_confusion_matrix')

In [ ]:
plot_confusion_matrix_manual(untrained_model_stats_person[0], untrained_model_stats_person[1], untrained_model_stats_person[2], untrained_model_stats_person[3], 'untrained_person_confusion_matrix')

In [ ]:
no_augmentation_model = load_model('Without_Augmentations')
no_augmentation_model.train()

if not os.path.exists('best_nms_and_result_no_augmentation_tree.pkl'):
    best_nms_no_augmentation_tree = None
    best_result_no_augmentation_tree = 0

    for t in [0.05 * i for i in range(0, 21)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset(no_augmentation_model, score_threshold=0, device=best_config['device'],
                                                   for_tree=True, nms=t)
        if current_score > best_result_no_augmentation_tree:
            best_result_no_augmentation_tree = current_score
            best_nms_no_augmentation_tree = t
    with open('best_nms_and_result_no_augmentation_tree.pkl', 'wb+') as f:
        pickle.dump((best_nms_no_augmentation_tree, best_result_no_augmentation_tree), f)
else:
    with open('best_nms_and_result_no_augmentation_tree.pkl', 'rb') as f:
        best_nms_no_augmentation_tree, best_result_no_augmentation_tree = pickle.load(f)
print(f'{best_nms_no_augmentation_tree=}, {best_result_no_augmentation_tree=}')

if not os.path.exists('best_threshold_and_result_no_augmentation_tree.pkl'):
    best_threshold_no_augmentation_tree = None
    best_result_no_augmentation_tree = 0

    for t in [0.05 * i for i in range(0, 11)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset(no_augmentation_model, score_threshold=t, device=best_config['device'],
                                                   for_tree=True, nms=best_nms_no_augmentation_tree)
        if current_score > best_result_no_augmentation_tree:
            best_result_no_augmentation_tree = current_score
            best_threshold_no_augmentation_tree = t
    with open('best_threshold_and_result_no_augmentation_tree.pkl', 'wb+') as f:
        pickle.dump((best_threshold_no_augmentation_tree, best_result_no_augmentation_tree), f)
else:
    with open('best_threshold_and_result_no_augmentation_tree.pkl', 'rb') as f:
        best_threshold_no_augmentation_tree, best_result_no_augmentation_tree = pickle.load(f)
print(f'{best_threshold_no_augmentation_tree=}, {best_result_no_augmentation_tree=}')

In [ ]:
if not os.path.exists('best_nms_and_result_no_augmentation_person.pkl'):
    best_nms_no_augmentation_person = None
    best_result_no_augmentation_person = 0

    for t in [0.05 * i for i in range(0, 21)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset(no_augmentation_model, score_threshold=0, device=best_config['device'],
                                                   for_tree=False, nms=t)
        if current_score > best_result_no_augmentation_person:
            best_result_no_augmentation_person = current_score
            best_nms_no_augmentation_person = t
    with open('best_nms_and_result_no_augmentation_person.pkl', 'wb+') as f:
        pickle.dump((best_nms_no_augmentation_person, best_result_no_augmentation_person), f)
else:
    with open('best_nms_and_result_no_augmentation_person.pkl', 'rb') as f:
        best_nms_no_augmentation_person, best_result_no_augmentation_person = pickle.load(f)
print(f'{best_nms_no_augmentation_person=}, {best_result_no_augmentation_person=}')

if not os.path.exists('best_threshold_and_result_no_augmentation_person.pkl'):
    best_threshold_no_augmentation_person = None
    best_result_no_augmentation_person = 0

    for t in [0.05 * i for i in range(0, 11)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset(no_augmentation_model, score_threshold=t, device=best_config['device'],
                                                   for_tree=False, nms=best_nms_no_augmentation_person)
        if current_score > best_result_no_augmentation_person:
            best_result_no_augmentation_person = current_score
            best_threshold_no_augmentation_person = t
    with open('best_threshold_and_result_no_augmentation_person.pkl', 'wb+') as f:
        pickle.dump((best_threshold_no_augmentation_person, best_result_no_augmentation_person), f)
else:
    with open('best_threshold_and_result_no_augmentation_person.pkl', 'rb') as f:
        best_threshold_no_augmentation_person, best_result_no_augmentation_person = pickle.load(f)
print(f'{best_threshold_no_augmentation_person=}, {best_result_no_augmentation_person=}')

In [ ]:
plot_confusion_matrix_manual(best_model_stats_tree[0], best_model_stats_tree[1], best_model_stats_tree[2], best_model_stats_tree[3], 'best_tree_confusion_matrix')

In [ ]:
plot_confusion_matrix_manual(best_model_stats_person[0], best_model_stats_person[1], best_model_stats_person[2], best_model_stats_person[3], 'best_person_confusion_matrix')

In [ ]:
if not os.path.exists('no_augmentation_model_stats_tree.pkl'):
    with open('no_augmentation_model_stats_tree.pkl', 'wb+') as f:
        pickle.dump(get_stats(no_augmentation_model, get_art_dataset(check_ids_tree), torch.device('cuda'), best_threshold_no_augmentation_tree, True, best_nms_no_augmentation_tree), f)
with open('no_augmentation_model_stats_tree.pkl', 'rb') as f:
    no_augmentation_model_stats_tree = pickle.load(f)
if not os.path.exists('no_augmentation_model_stats_person.pkl'):
    with open('no_augmentation_model_stats_person.pkl', 'wb+') as f:
        pickle.dump(get_stats(no_augmentation_model, get_art_dataset(check_ids_person), torch.device('cuda'), best_threshold_no_augmentation_person, False, best_nms_no_augmentation_person), f)
with open('no_augmentation_model_stats_person.pkl', 'rb') as f:
    no_augmentation_model_stats_person = pickle.load(f)

In [ ]:
plot_confusion_matrix_manual(no_augmentation_model_stats_tree[0], no_augmentation_model_stats_tree[1], no_augmentation_model_stats_tree[2], no_augmentation_model_stats_tree[3], 'no_augmentation_tree_confusion_matrix')

In [ ]:
plot_confusion_matrix_manual(no_augmentation_model_stats_person[0], no_augmentation_model_stats_person[1], no_augmentation_model_stats_person[2], no_augmentation_model_stats_person[3], 'no_augmentation_person_confusion_matrix')

In [ ]:
import torchvision
del no_augmentation_model
gc.collect()
coco_model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
coco_model.to(torch.device('cuda'))

def get_stats_coco(from_model, ds, device, score_threshold, nms=0.1):
    check_ids = check_ids_person
    from_model.to(device)
    from_model.eval()
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    pos = 0
    neg = 0
    identifiers_false_positive = []
    identifiers_false_negative = []
    for ident in (loading_bar := tqdm(check_ids, total=len(check_ids))):
        stats_img, _ = ds.get_item_by_identifier(ident)
        stats_output = from_model([stats_img.to(device)])
        stats_reduced = reduce_box_count(stats_output[0], nms, score_threshold, 0)
        stats_labels = stats_reduced['labels']

        del stats_img
        del stats_output
        del stats_reduced
        gc.collect()

        found = 1 in stats_labels
        exists = da.get_person_annotation_for_identifier(ident)

        if exists:
            pos += 1
            if found:
                true_positive += 1
            else:
                false_negative += 1
                identifiers_false_negative.append(ident)
        else:
            neg += 1
            if found:
                false_positive += 1
                identifiers_false_positive.append(ident)
            else:
                true_negative += 1
        loading_bar.set_description(
            f'{true_positive=}, {false_positive=}, {true_negative=}, {false_negative=}, {pos=}, {neg=}')
    return true_positive, false_positive, true_negative, false_negative, pos, neg, identifiers_false_positive, identifiers_false_negative

def evaluate_model_for_dataset_coco(model_to_evaluate, score_threshold, device, nms):
    check_ids = check_ids_person
    stats = get_stats_coco(model_to_evaluate, get_art_dataset(check_ids), device, score_threshold, nms=nms)
    true_positive = stats[0]
    false_positive = stats[1]
    false_negative = stats[3]
    if (true_positive + false_positive) == 0 or (true_positive + false_negative) == 0:
        return 0
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    if precision + recall == 0:
        return 0
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


if not os.path.exists('best_nms_and_result_coco_person.pkl'):
    best_nms_coco_person = None
    best_result_coco_person = 0

    for t in [0.05 * i for i in range(0, 21)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset_coco(coco_model, score_threshold=0, device=best_config['device'], nms=t)
        if current_score > best_result_coco_person:
            best_result_coco_person = current_score
            best_nms_coco_person = t
    with open('best_nms_and_result_coco_person.pkl', 'wb+') as f:
        pickle.dump((best_nms_coco_person, best_result_coco_person), f)
else:
    with open('best_nms_and_result_coco_person.pkl', 'rb') as f:
        best_nms_coco_person, best_result_coco_person = pickle.load(f)
print(f'{best_nms_coco_person=}, {best_result_coco_person=}')

if not os.path.exists('best_threshold_and_result_coco_person.pkl'):
    best_threshold_coco_person = None
    best_result_coco_person = 0

    for t in [0.05 * i for i in range(0, 11)]:
        print(f'testing with: {t}')
        current_score = evaluate_model_for_dataset_coco(coco_model, score_threshold=t, device=best_config['device'], nms=best_nms_coco_person)
        if current_score > best_result_coco_person:
            best_result_coco_person = current_score
            best_threshold_coco_person = t
    with open('best_threshold_and_result_coco_person.pkl', 'wb+') as f:
        pickle.dump((best_threshold_coco_person, best_result_coco_person), f)
else:
    with open('best_threshold_and_result_coco_person.pkl', 'rb') as f:
        best_threshold_coco_person, best_result_coco_person = pickle.load(f)
print(f'{best_threshold_coco_person=}, {best_result_coco_person=}')

In [ ]:
if not os.path.exists('coco_model_stats_person.pkl'):
    with open('coco_model_stats_person.pkl', 'wb+') as f:
        pickle.dump(get_stats_coco(coco_model, get_art_dataset(check_ids_person), torch.device('cuda'), best_threshold_coco_person, best_nms_coco_person), f)
with open('coco_model_stats_person.pkl', 'rb') as f:
    coco_model_stats_person = pickle.load(f)

In [ ]:
plot_confusion_matrix_manual(coco_model_stats_person[0], coco_model_stats_person[1], coco_model_stats_person[2], coco_model_stats_person[3], 'coco_person_confusion_matrix')

In [ ]:
del coco_model
gc.collect()

unaugmented_results = dict()
augmented_results = dict()

def get_mcnemars_matrices():
    model_name_1 = 'Without_Augmentations'
    model_name_2 = 'With_Augmentations'

    results = {'Tree': {model_name_1: dict(), model_name_2: dict()}, 'Person': {model_name_1: dict(), model_name_2: dict()}}

    for model_name in [model_name_1, model_name_2]:
        current_model = load_model(model_name)
        current_model.to(torch.device('cuda'))
        current_model.eval()
        for for_tree in [True, False]:
            if for_tree:
                check_ids = check_ids_tree
                current_dictionary = results['Tree']
            else:
                check_ids = check_ids_person
                current_dictionary = results['Person']

            mc_neamar_dataset = get_art_dataset(check_ids)

            for ident in tqdm(check_ids, total=len(check_ids)):
                mc_neamar_img, mc_neamar_target = mc_neamar_dataset.get_item_by_identifier(ident)
                mc_neamar_output = current_model([mc_neamar_img.to(torch.device('cuda'))])

                if model_name == 'With_Augmentations':
                    if for_tree:
                        ideal_nms = best_nms_tree
                        ideal_threshold = best_threshold_tree
                    else:
                        ideal_nms = best_nms_person
                        ideal_threshold = best_threshold_person
                else:
                    if for_tree:
                        ideal_nms = best_nms_no_augmentation_tree
                        ideal_threshold = best_threshold_no_augmentation_tree
                    else:
                        ideal_nms = best_nms_no_augmentation_person
                        ideal_threshold = best_threshold_no_augmentation_person
                mc_neamar_reduced = reduce_box_count(mc_neamar_output[0], ideal_nms, ideal_threshold, 0)
                mc_neamar_labels = [da.get_class_label_for_index(l) for l in mc_neamar_reduced['labels']]

                if for_tree:
                    found = 'Baum' in mc_neamar_labels
                    exists = da.get_tree_annotation_for_identifier(ident)
                else:
                    found = 'Person' in mc_neamar_labels
                    exists = da.get_person_annotation_for_identifier(ident)

                if exists:
                    if found:
                        current_dictionary[model_name][ident] = 'TP'
                    else:
                        current_dictionary[model_name][ident] = 'FN'
                else:
                    if found:
                        current_dictionary[model_name][ident] = 'FP'
                    else:
                        current_dictionary[model_name][ident] = 'TN'
        del current_model
        gc.collect()

    matrices = {'Tree': [[0,0], [0,0]], 'Person': [[0,0], [0,0]]}
    correct_results = ['TP', 'TN']

    for test in ['Tree', 'Person']:
        if test == 'Tree':
            check_ids = check_ids_tree
        else:
            check_ids = check_ids_person

        for ident in check_ids:
            result_1 = results[test][model_name_1][ident]
            result_2 = results[test][model_name_2][ident]
            if result_1 in correct_results and result_2 in correct_results:
                matrices[test][0][0] += 1
            elif result_1 in correct_results and result_2 not in correct_results:
                matrices[test][0][1] += 1
            elif result_1 not in correct_results and result_2 in correct_results:
                matrices[test][1][0] += 1
            elif result_1 not in correct_results and result_2 not in correct_results:
                matrices[test][1][1] += 1
            else:
                raise Exception('Unhandled Case')

    matrices['Both'] = [[matrices['Tree'][0][0] + matrices['Person'][0][0], matrices['Tree'][0][1] + matrices['Person'][0][1]], [matrices['Tree'][1][0] + matrices['Person'][1][0], matrices['Tree'][1][1] + matrices['Person'][1][1]]]

    return matrices

if os.path.exists(path := 'mcnemar_matrices.pkl'):
    with open(path, 'rb') as f:
        mcnemar_matrices = pickle.load(f)
else:
    mcnemar_matrices = get_mcnemars_matrices()
    with open(path, 'wb+') as f:
        pickle.dump(mcnemar_matrices, f)

In [ ]:
print(mcnemar(mcnemar_matrices['Person'], exact=False))

The null hypothesis which claims that the two model's performances are equal can be rejected (p value < 0.05)

In [ ]:
print(mcnemar(mcnemar_matrices['Tree'], exact=False))